# **Proyecto Statistical Learning**
# Primera parte
**José Barrios - 20007192** 

## Descripción general
El proyecto consiste en hacer una  clasificación binaria para determinar si una persona sobrevive (y=1), o no (y=0), el hundimiento del Titanic.

Se busca crear un modelo con una exactitud de al menos el 80%. 

El proyecto está dividido en dos partes. En esta primera parte nos enfocaremos en crear los modelos, evaluar su desempeño y guardarlos para ser posteriormente usados en otro notebook.

En general, se hará _feature engineering_ y luego el entrenamiento de 4 modelos:
* Árbol de decisión
* Support Vector Machine
* Naive Bayes
* Regresión Logística
Los resultados de estos modelos se combinarán y ayudarán a predecir la supervivencia de una persona en concenso.

## Datos
Obtenemos los datos para concer su estructura.

In [277]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import sklearn.preprocessing as skp 
from sklearn import tree
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import datetime

In [109]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Enabled compatitility to tf1.x


In [168]:
%load_ext tensorboard

In [63]:
data = pd.read_csv('data_titanic_proyecto.csv')
data.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N


In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
PassengerId     891 non-null int64
Age             891 non-null float64
SibSp           891 non-null int64
Parch           891 non-null int64
Fare            891 non-null float64
Embarked_C      891 non-null uint8
Embarked_Q      891 non-null uint8
Embarked_S      891 non-null uint8
Class_Lower     891 non-null uint8
Class_Middle    891 non-null uint8
Class_Upper     891 non-null uint8
Sex_F           891 non-null uint8
Sex_M           891 non-null uint8
Survived        891 non-null int64
dtypes: float64(2), int64(4), uint8(8)
memory usage: 48.9 KB


Notamos que algunas columnas las podemos omitir para realizar nuestros análisis ya que se considera que no aportarán a los modelos:
* Name
* Ticket
* Cabin

Cabin tambien se eliminó porque tiene demasiados valores nulos cuando podemos obtener información de otras features.

In [64]:
data = data.drop(columns = ["Name", "Ticket", "Cabin"])
data.head()

,PassengerId,Age,SibSp,Parch,Fare,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,22.0,1,0,7.2500,S,Lower,M,N
1,2,38.0,1,0,71.2833,C,Upper,F,Y
2,3,26.0,0,0,7.9250,S,Lower,F,Y
3,4,35.0,1,0,53.1000,S,Upper,F,Y
4,5,35.0,0,0,8.0500,S,Lower,M,N


Las edades nulas las reemplazamos con la media de la edad del dataset.

In [73]:
data["Age"] = data["Age"].fillna(data["Age"].mean())

Realizamos one-hot-encoding para las variables Embarked, Passenger class y Passenger sex

In [65]:
data = pd.concat([data, pd.get_dummies(data["Embarked"], prefix = "Embarked")], axis = 1)
data = pd.concat([data, pd.get_dummies(data["passenger_class"], prefix = "Class")], axis = 1)
data = pd.concat([data, pd.get_dummies(data["passenger_sex"], prefix = "Sex")], axis = 1)

data = data.drop(columns = ["Embarked", "passenger_class", "passenger_sex"])
data.head()

,PassengerId,Age,SibSp,Parch,Fare,passenger_survived,Embarked_C,Embarked_Q,Embarked_S,Class_Lower,Class_Middle,Class_Upper,Sex_F,Sex_M
0,1,22.0,1,0,7.2500,N,0,0,1,1,0,0,0,1
1,2,38.0,1,0,71.2833,Y,1,0,0,0,0,1,1,0
2,3,26.0,0,0,7.9250,Y,0,0,1,1,0,0,1,0
3,4,35.0,1,0,53.1000,Y,0,0,1,0,0,1,1,0
4,5,35.0,0,0,8.0500,N,0,0,1,1,0,0,0,1


Convertimos passenger_survived a enteros donde Y = 1 y N = 0

In [66]:
data["Survived"] = 1
data.loc[data["passenger_survived"] == "N", "Survived"] = 0
data = data.drop(columns = ["passenger_survived"])
data.head()

,PassengerId,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Class_Lower,Class_Middle,Class_Upper,Sex_F,Sex_M,Survived
0,1,22.0,1,0,7.2500,0,0,1,1,0,0,0,1,0
1,2,38.0,1,0,71.2833,1,0,0,0,0,1,1,0,1
2,3,26.0,0,0,7.9250,0,0,1,1,0,0,1,0,1
3,4,35.0,1,0,53.1000,0,0,1,0,0,1,1,0,1
4,5,35.0,0,0,8.0500,0,0,1,1,0,0,0,1,0


Aprovechamos Pandas para crear los datasets de entrenamiento, pruebas, y validación.

In [76]:
data_train = data.sample(frac = 0.8, random_state = 123) #random state es un random seed
data_test = data.drop(data_train.index) #Data de testing
data_cv = data_train.sample(frac = 0.12, random_state = 123) #Data de validación (12% del entrenamiento, aprox. 10% del set original)
data_train = data_train.drop(data_cv.index) #Data entrenamiento final, ya sin la de testing ni cross-validation

print(data_train.shape)
print(data_test.shape)
print(data_cv.shape)

(627, 14)
(178, 14)
(86, 14)


## Creación de modelos
### Árbol de decisión
Implementación de árbol de decisión con ScikitLearn. Se saca provecho de que los árboles de decisión necesitan poca feature engineering para realizar un entrenamiento, así que únicamente haremos uso de las librerías disponibles.

In [243]:
model_tree = tree.DecisionTreeClassifier()
model_tree = model_tree.fit(skp.scale(data_train[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]), 
                            data_train["Survived"])

In [244]:
prediccion_tree = model_tree.predict(skp.scale(data_cv[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]))

Se procede a almacenar el árbol para poder utilizarlo en otro proyecto.

In [232]:
tree.export_graphviz(model_tree, 
                     out_file = "models/model_tree.dot", 
                     feature_names = ["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"],
                     filled=True, 
                     rounded=True,
                     special_characters=True)

Definimos una función que almacena los experimentos en una hoja de Excel

In [ ]:
#from openpyxl import load_workbook
#with pd.ExcelWriter('bitacora.xlsx', engine='openpyxl', mode='a') as writer:
#    data_cv.to_excel(writer,sheet_name='Sheet1')
#    writer.save()

#writer.close()

### Support Vector Machine (SVM)
Es un método de aprendizaje supervizado que es especialmente últil, aunque no limitado, para:
* Efetivo para espacios altamente dimensionales
* Efectivo, en su mayoría, aunque el número de dimensiones es mayor al número de observaciones
* Utiliza vectores de soporte, que son eficientes en el uso de memoria

Este modelo también se implementará utilizando Scikitlearn.

In [233]:
model_svm = svm.SVC()
model_svm = model_svm.fit(skp.scale(data_train[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]), 
                          data_train["Survived"])


In [234]:
prediccion_svm = model_svm.predict(skp.scale(data_cv[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]))

Se almacena el modelo SVM para su posterior uso en otro proyecto.

### Naive Bayes
Implementación de Bayes con Pandas. 

In [236]:
model_bayes = GaussianNB()
model_bayes = model_bayes.fit(skp.scale(data_train[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]), 
                              data_train["Survived"])

In [239]:
prediccion_bayes = model_bayes.predict(skp.scale(data_cv[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]))

Almacenamos los parámetros del modelo para su posterior uso en otro proyecto.

### Regresión Logística
Implementación con TensorFlow para un algoritmo con regresión logística usando la función Sigmoid con regularización tipo Lasso.

Obtenemos un grafo de la forma:

<img src="images/SigmoidGraph.png">

In [169]:
class SigmoidClasification:
    def __init__(self, features):
        self.theta = tf.get_variable(shape = [features, 1], 
                                 dtype = tf.float32, 
                                 initializer = tf.zeros_initializer(),
                                 name = "theta")
    
    def logits(self, x):
        return tf.matmul(x, self.theta)
    
    def prediction(self, x):
        return tf.math.sigmoid(self.logits(x))
    
    def accuracy(self, predictions, labels):
        _, accuracy = tf.metrics.accuracy(labels = tf.argmax(labels, 1),
                                          predictions = tf.argmax(predictions, 1))
        return accuracy
    
    def update(self, x_train, labels_train, learning_rate):
        with tf.name_scope("cross_entropy"):
            train_error = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = labels_train, 
                                                                                 logits = self.logits(x_train)))
            train_error_summary = tf.summary.scalar("train_error", train_error)
            
        with tf.name_scope("update"):
            gradient = tf.gradients(train_error, [self.theta])
            theta_new = tf.assign(self.theta, self.theta - learning_rate * gradient[0])
            
        return theta_new, train_error, train_error_summary

In [170]:
def training(learning_rate, epochs, imprimir_error_cada, x_train, labels_train, batch_size = 32):
    log = './logs/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_lr=" +str(learning_rate) + "_epochs=" + str(epochs)
    g = tf.Graph()
    
    with g.as_default():
        modelo = SigmoidClasification(x_train.shape[1])
        
        with tf.name_scope("train_tensor"):
            tensor_x_train = tf.placeholder(tf.float32, [None, x_train.shape[1]], "tensor_x_train")
            tensor_labels_train = tf.placeholder(tf.float32, [None, 1], "tensor_labels_train")
        
        update_parameters = modelo.update(tensor_x_train, tensor_labels_train, learning_rate)
        writer = tf.summary.FileWriter(log, g)
        
        total_steps = int((labels_train.shape[0] / batch_size) * epochs)  
        
        with tf.train.MonitoredSession() as session:
            for i in range(total_steps + 1):
                offset = (i * batch_size) % (x_train.shape[0] - batch_size)
                
                batch_data = x_train[offset : (offset + batch_size),]
                batch_labels = labels_train[offset : (offset + batch_size)]
                
                parameters_dict = {tensor_x_train: batch_data,
                                   tensor_labels_train: batch_labels}
                training = session.run(update_parameters, feed_dict = parameters_dict)
                
                if (i % imprimir_error_cada == 0):
                    theta = session.run(modelo.theta, feed_dict = parameters_dict)
                    #print("Epoch ", (i % labels_train.shape[0]) + 1 , ": Pesos: ", theta, "Cost: ", training[1])
                    writer.add_summary(training[2], i)
            
            return theta

            writer.close()

In [189]:
y = data_train[["Survived"]].to_numpy()
x = data_train[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]
x["Bias"] = 1
x = skp.scale(x)
x.shape

D:\Programs\Anaconda\envs\clase22\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(627, 13)

In [190]:
theta = training(0.01, 50, 5, x, y)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Durante el entrenamiento, los experimentos con features escalados generaban menor error de entrenamiento y podemos notarlo en la diferencia de escala en esta gráfica donde las lineas superiores son la data cruda y las inferiores son con datos escalados.

<img src="images/SigmoidTrainingAll.png">

Ahora se muestran los errores de los entrenamientos de datos escalados.

<img src="images/SigmoidTrainingScaled.png">


In [173]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 8744), started 0:00:59 ago. (Use '!kill 8744' to kill it.)

Creamos una clase que reciba los pesos y arme un modelo de regresión logística con ellos.

Para esta clase asumimos que el orden las features viene del siguiente modo:
1. Age
2. SibSp
3. Parch
4. Fare
5. Embarked_C
6. Embarked_Q
7. Embarked_S
8. Class_Lower
9. Class_Middle
10. Class_Upper
11. Sex_F
12. Sex_M
13. Bias



In [191]:
class SigmoidModel:
    def __init__(self, theta):
        self.theta = theta
    
    def logits(self, x):
        return tf.matmul(x, self.theta)
    
    def prediction(self, x):
        return tf.math.sigmoid(self.logits(x))
    
    def accuracy(self, predictions, labels):
        _, accuracy = tf.metrics.accuracy(labels = tf.argmax(labels, 1),
                                          predictions = tf.argmax(predictions, 1))
        return accuracy

In [216]:
def Sigmoidprediction(theta, x):
    g = tf.Graph()
    
    with g.as_default():
        modelo = SigmoidModel(theta)
        
        tensor_x = tf.placeholder(tf.float32, [None, x.shape[1]], "tensor_x")
        prediction = modelo.prediction(tensor_x)
        
        with tf.train.MonitoredSession() as session:
            parameters_dict = {tensor_x: x}
            data = session.run(prediction, feed_dict = parameters_dict)
            result = np.where(data < 0.5, 0, 1) 
            result = result.reshape(-1, )
            return result

In [193]:
x_cv = data_cv[["Age", "SibSp", "Parch", "Fare", "Embarked_C", "Embarked_Q", "Embarked_S", "Class_Lower", "Class_Middle", "Class_Upper", "Sex_F", "Sex_M"]]
x_cv["Bias"] = 1
x_cv = skp.scale(x_cv)
x_cv.shape

D:\Programs\Anaconda\envs\clase22\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(86, 13)

In [217]:
prediccion_sigmoid = Sigmoidprediction(theta, x_cv)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


## Ensemble Learning
Ya tenemos resultados de los 4 modelos, así que con la data de validación creamos un reporte para determinar la exactitud de las predicciones en concenso.

Primero se fusionan los resultados en un solo.

In [273]:
resultados = pd.DataFrame({
    #"PassengerId": data_cv["PassengerId"], 
    "Tree": prediccion_tree, 
    "SVM": prediccion_svm, 
    "Bayes": prediccion_bayes, 
    "Sigmoid": prediccion_sigmoid})
resultados.head()

,Tree,SVM,Bayes,Sigmoid
0,1,1,1,1
1,0,0,0,0
2,1,1,1,1
3,0,1,1,1
4,0,0,0,1


Usaremos el promedio de la suma de los resultados y si es mayor a 0.5, se dirá que sobrevivió.

In [276]:
resultados['Survived'] = resultados.mean(numeric_only=True, axis=1)
resultados["Predicted"] = np.where(resultados['Survived'] < 0.5, 0, 1)
resultados = resultados.drop(["Survived"], axis=1)
resultados["Real"] = data_cv[["Survived"]].to_numpy()
resultados.head()

,Tree,SVM,Bayes,Sigmoid,Predicted,Real
0,1,1,1,1,1,1
1,0,0,0,0,0,0
2,1,1,1,1,1,1
3,0,1,1,1,1,1
4,0,0,0,1,0,0


In [280]:
print("Precisión de validación")
print("Precisión de Tree: ", accuracy_score(resultados["Tree"], resultados["Real"]))
print("Precisión de SVM: ", accuracy_score(resultados["SVM"], resultados["Real"]))
print("Precisión de Bayes: ", accuracy_score(resultados["Bayes"], resultados["Real"]))
print("Precisión de Sigmoid: ", accuracy_score(resultados["Sigmoid"], resultados["Real"]))
print("Precisión de Ensemble: ", accuracy_score(resultados["Predicted"], resultados["Real"]))

Precisión de validación
Precisión de Tree:  0.6162790697674418
Precisión de SVM:  0.8604651162790697
Precisión de Bayes:  0.7906976744186046
Precisión de Sigmoid:  0.813953488372093
Precisión de Ensemble:  0.7906976744186046


## Conclusión
* El modelo con el peor desempeño fue el de árbol de decisión. Para este caso, quizas hubiera ayudado realizar random forest en vez de un solo árbol. Sin embargo, ese modelo tiene la ventaja de ser "facil de explicar" al negocio.
* La utilización de escalado para realizar feature engineering a los datos de training, y a los de test, disminuyen el error durante la fase de entrenamiento. Se pudo notar que para estos datos la variación del error es más baja y su media también.
* Ensemble learning ayuda a complementar las predicciones de varios modelos y apoyarse en las ventajas y complementar posibles contras de cada uno de ellos.

## K-Folds Cross-Validation
Es una técnica de remuestreo que consiste en realizar k muestras aleatorias con la data de entrenamiento y utilizar una de esas k muestras para validación en cada epoch. En general, el proceso es el siguiente:
1. Mezclar las observaciones aleatoriamente
2. Separar la data en k grupos
3. Para cada grupo, hacer una vez:
    * Tomar el grupo y considerarlo como data de testing
    * Los demáß grupos serán data de entrenamiento
    * Entrenar el modelo y evaluarlo con el grupo que separamos para testing
    * Retener la métrica de desempeño y descartar el modelo
4. Sintetizar el desempeño de los modelos

Se debe destacar que cada observación queda fija en el grupo que quedó durante el remuestreo de k grupos, esto quiere decir que cada observación será utilizada una vez como validación y k-1 veces como entrenamiento.

<img src="images/cross.png" alt="Ejemplo k=5">

Para este proyecto se podría haber utilizado para validar iterativamente los modelos al mismo tiempo que se entrenan, además de mostrar el desempeño que tiene cada vez que realice las validaciones.